# Imports and variables

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
from naludaq.board import Board
from naludaq.communication import (
    ControlRegisters,
    DigitalRegisters,
    AnalogRegisters,
)

### Logger

In [ ]:
from logging import getLogger, Formatter, StreamHandler, INFO, DEBUG, ERROR

def setup_logger(level=DEBUG):
    logger = getLogger()
    handler = StreamHandler()
    handler.setFormatter(Formatter('%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s'))
    logger.addHandler(handler)
    logger.setLevel(level)
    uart = getLogger('naludaq.board.connections._UART')
    uart.setLevel(ERROR)
    
    return logger

try:
    logger.info("logger already setup")
except:
    logger = setup_logger()
    
uart = getLogger('naludaq.board.connections._UART')
uart.setLevel(DEBUG)

## Board creation

In [ ]:
MODEL = "hiper"
SERIAL = "EA8C7B"
BAUDRATE = 3_000_000

In [ ]:
BOARD = Board(MODEL)

In [ ]:
from naludaq.tools.ftdi import list_ftdi_devices

In [ ]:
list_ftdi_devices()

In [ ]:
BOARD.get_ftdi_connection(serial_number=SERIAL, baud=BAUDRATE)
BOARD.disconnect()

In [ ]:
CR = ControlRegisters(BOARD)
DR = DigitalRegisters(BOARD)

# Board control

## Multichip control
Since the board has multiple chips, we need to be able to select which chip to program.
However we can't read the chips

### Read/Write control registers

In [ ]:
# Hiper needs to have installed chips specified as a board param.
with BOARD:
    CR.read('identifier')

In [ ]:
with BOARD:
    print(f"{CR.read('busy_locked')}")
    print(f"{CR.read('idle_det')}")

In [ ]:
expected_val = int(b'1010101', 2)
with BOARD:
    CR.read('runevs')
    time.sleep(0.1)
    CR.write('runevs', expected_val)
    time.sleep(0.1)
    print(CR.read('runevs')["value"] == expected_val)

### Read/Write to digital registers

In [ ]:
# THIS IS BROKEN! YOU CAN'T READ DIGITAL REGISTERS WITH THE HIPER

#####################################
# TXIn Enable OFF TO AVOID UNWANTED RETURN DATA.
# CAN BE ON IF YOU WANT RETURN DATA FOR EACH COMMAND

#####################################
# select the first chip
backup_rx = CR.read('rxout_en')["value"]
backup_tx = CR.read('txin_en')["value"]
with BOARD:
    CR.write('rxout_en', 65535)
    time.sleep(0.1)
    CR.write('txin_en', 65535) # int('1110' '000' '000' '000' '001',2))
    time.sleep(0.1)

    DR.read('chipid')
    DR.write('chipid', 0b10101010)
    print(DR.read('chipid')["value"] == 0b10101010)

    time.sleep(0.1)
    CR.write('rxout_en', backup_rx)
    CR.write('txin_en', backup_tx)

### Startup sequence

In [ ]:
from naludaq.board import startup_board

In [ ]:
started = startup_board(BOARD)
assert started == True